# 03 Daft + LanceDB 集成

本 Notebook 展示 Daft 与 LanceDB 的集成使用：
- 用 Daft 读取 Lance 格式数据（LanceDB 底层存储）
- 用 Daft 做数据预处理
- 用 Daft 内置的 `embed_text` 批量生成嵌入
- 将结果写入 LanceDB

**前置要求**：
- 已运行 Notebook 02（生成了 `lancedb_data/reviews.lance`）
- 已配置 SiliconFlow API Key

## 1. Daft 读取 Lance 格式

LanceDB 底层使用 Lance 列式存储格式。Daft 可以通过 `daft.read_lance()` 直接读取，无需经过 LanceDB API。

这意味着你可以用 Daft 的 DataFrame API 对 LanceDB 中的数据做分析和处理。

In [ ]:
import daft
from daft import col

# 读取 Notebook 02 写入的 reviews 表（Lance 格式）
lance_path = "../lancedb_data/reviews.lance"
df_lance = daft.read_lance(lance_path)

print(f"Schema:")
print(df_lance.schema())
print(f"\n前 5 条:")
df_lance.select("cat", "label", "review").limit(5).show()

### 用 Daft 分析 LanceDB 数据

直接在 Lance 数据上做聚合统计，无需加载到内存。

In [ ]:
# 按类别统计评论数和正面评论占比
stats = (
    df_lance
    .groupby("cat")
    .agg(
        col("review").count().alias("count"),
        col("label").mean().alias("positive_ratio"),
    )
    .sort("count", desc=True)
)
stats.show()

## 2. 用 Daft 做数据预处理

从原始 CSV 开始，用 Daft 做清洗和过滤，为嵌入生成做准备。

In [ ]:
# 读取原始 CSV
df_raw = daft.read_csv("../data/reviews.csv")
print(f"原始数据:")
df_raw.limit(3).show()

In [ ]:
# 数据预处理：过滤空评论、添加评论长度列
df_clean = (
    df_raw
    .where(col("review").not_null())
    .where(col("review").str.length() > 5)  # 过滤过短的评论
    .with_column(
        "review_length",
        col("review").str.length(),
    )
)

print(f"清洗后数据:")
df_clean.limit(3).show()

In [ ]:
# 统计清洗结果
print("各类别评论数:")
df_clean.groupby("cat").agg(col("review").count().alias("count")).sort("count", desc=True).show()

print("\n评论长度统计:")
df_clean.select(
    col("review_length").min().alias("min_len"),
    col("review_length").mean().alias("avg_len"),
    col("review_length").max().alias("max_len"),
).show()

## 3. 用 Daft embed_text 生成嵌入

Daft 内置了 `embed_text` 函数，可以直接在 DataFrame 上批量生成嵌入向量。配合 `set_provider` 设置 API 提供商。

相比 Notebook 02 中手动调用 OpenAI SDK，Daft 的方式更简洁，且支持 lazy evaluation 和并行执行。

In [ ]:
import os
from daft.functions.ai import embed_text

# 配置 SiliconFlow 作为 OpenAI 兼容提供商
assert os.environ.get("OPENAI_API_KEY"), "请设置环境变量 OPENAI_API_KEY"

daft.set_provider(
    "openai",
    base_url=os.environ.get("OPENAI_BASE_URL", "https://api.siliconflow.cn/v1"),
)
print(f"Provider: {daft.current_provider()}")

### 为小批量数据生成嵌入（演示）

先用少量数据验证 pipeline 是否正常工作。

In [ ]:
EMBEDDING_MODEL = "Qwen/Qwen3-Embedding-4B"
EMBEDDING_DIM = 1024

# 取 10 条数据做演示
df_sample = df_clean.limit(10)

# 用 embed_text 生成嵌入向量
df_with_vec = df_sample.with_column(
    "vector",
    embed_text(
        col("review"),
        model=EMBEDDING_MODEL,
        dimensions=EMBEDDING_DIM,
    ),
)

# 查看结果
df_with_vec.select("cat", "review", "vector").show()

## 4. 完整 Pipeline：Daft 预处理 → 嵌入 → 写入 LanceDB

将上述步骤串联为完整的 pipeline：
1. 读取 CSV
2. 清洗过滤
3. 生成嵌入
4. 写入 LanceDB
5. 语义搜索验证

In [ ]:
# 完整 pipeline（使用全量数据）
df_pipeline = (
    daft.read_csv("../data/reviews.csv")
    .where(col("review").not_null())
    .where(col("review").str.length() > 5)
    .with_column(
        "vector",
        embed_text(
            col("review"),
            model=EMBEDDING_MODEL,
            dimensions=EMBEDDING_DIM,
        ),
    )
)

# 收集结果到 pandas（触发实际计算）
print("正在执行 pipeline（读取 → 清洗 → 嵌入生成）...")
df_result = df_pipeline.to_pandas()
print(f"完成，共 {len(df_result)} 条记录，向量维度 {len(df_result['vector'].iloc[0])}")

In [ ]:
import lancedb

# 写入 LanceDB
db = lancedb.connect("../lancedb_data")
table = db.create_table("reviews_daft", df_result.to_dict("list"), mode="overwrite")

print(f"表名: {table.name}")
print(f"行数: {table.count_rows()}")
print(f"Schema: {table.schema}")

### 验证：语义搜索

在 Daft pipeline 写入的数据上执行语义搜索，验证端到端流程。

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ.get("OPENAI_BASE_URL", "https://api.siliconflow.cn/v1"),
)


def search_reviews(query: str, top_k: int = 5):
    """语义搜索评论"""
    response = client.embeddings.create(input=[query], model=EMBEDDING_MODEL)
    query_vec: list[float] = response.data[0].embedding
    results = table.search(query_vec).limit(top_k).to_pandas()
    return results


# 测试搜索
queries = ["质量好，非常满意", "手机屏幕效果", "酒店服务差"]
for q in queries:
    print(f"\n查询: '{q}'")
    results = search_reviews(q, top_k=3)
    for _, row in results.iterrows():
        print(f"  [{row['cat']}] {row['review'][:50]}")

## 5. Daft write_lance：直接写入 Lance 格式

除了通过 LanceDB API 写入，Daft 还可以直接用 `write_lance()` 写入 Lance 格式文件。

这在不需要 LanceDB 索引和搜索功能、只需要高效列式存储时很有用。

In [ ]:
# 用 Daft 直接写入 Lance 格式
output_path = "../data/reviews_processed.lance"

df_for_lance = (
    daft.read_csv("../data/reviews.csv")
    .where(col("review").not_null())
    .where(col("review").str.length() > 5)
    .with_column("review_length", col("review").str.length())
)

df_for_lance.write_lance(output_path, mode="overwrite")
print(f"已写入 Lance 格式: {output_path}")

# 验证：用 Daft 读回
df_read_back = daft.read_lance(output_path)
df_read_back.limit(3).show()

## 6. 扩展：Ray Runner 加速大规模嵌入生成

当数据量达到十万甚至百万级别时，单机生成嵌入会成为瓶颈。Daft 支持 Ray 作为分布式执行引擎：

```python
# 切换到 Ray Runner（需要 Ray 集群）
daft.context.set_runner_ray()

# 同样的 pipeline 代码，自动分布式执行
df = (
    daft.read_csv("s3://bucket/reviews_large.csv")
    .with_column("vector", embed_text(col("review"), model=EMBEDDING_MODEL))
)
df.write_lance("s3://bucket/reviews.lance")
```

Ray Runner 的优势：
- 自动将数据分片到多个 worker 并行处理
- 支持 S3/GCS 等云存储
- 代码无需修改，只需切换 runner

详见 [Demo 2: Ray 分布式计算](../../demo2_ray/) 了解 Ray 的使用。

## 清理

In [ ]:
import shutil

# 清理演示生成的 Lance 文件
shutil.rmtree("../data/reviews_processed.lance", ignore_errors=True)
print("清理完成")

## 小结

本 Notebook 展示了 Daft 与 LanceDB 的集成：

- **读取 Lance 格式**：`daft.read_lance()` 直接读取 LanceDB 底层数据
- **数据预处理**：用 Daft DataFrame API 做清洗、过滤、统计
- **嵌入生成**：`embed_text()` 内置函数，配合 `set_provider` 使用 SiliconFlow API
- **写入 LanceDB**：通过 `to_pandas()` 中转写入 LanceDB 表
- **写入 Lance 格式**：`write_lance()` 直接写入 Lance 列式存储
- **扩展**：切换 Ray Runner 即可分布式处理大规模数据

这构成了一个完整的数据处理 pipeline：**数据读取 → 清洗 → 嵌入生成 → 向量存储 → 语义搜索**。